In [1]:
import property_inference_interface
interface = property_inference_interface.PropertyInferenceInterface()

meta_params = {
    'num_of_LPs': 4,
    'size_of_train_set': 10000,
    'size_of_test_set': 100,
    'flatten': False, 
    'model_type': 'CNN',
    'adv_attack': 'i_FGSM'
}

interface.set_meta_params(meta_params)
interface.print_meta_params()

num_of_LPs           : 4
size_of_train_set    : 10000
size_of_test_set     : 100
flatten              : False
model_type           : CNN
adv_attack           : i_FGSM


In [ ]:
interface.prepare_dataset()
interface.print_dataset_shape()

In [ ]:
# Visualize the dataset
%matplotlib inline
interface.print_some_samples()

In [ ]:
# interface.generate_model(num_of_epochs=15)
interface.load_model('MNIST_CNN.pt')
# interface.store_model('MNIST_CNN.pt')

interface.generate_robustified_model()
print('Display the architecture: original')
print(interface.model)
print('Display the architecture: robustified')
print(interface.robustified_model)

In [ ]:
print('Evaluate accurancy: original')
interface.eval_model('train', on_robustified_model=False)
interface.eval_model('test', on_robustified_model=False)
print('Evaluate accurancy: robustified')
interface.eval_model('train', on_robustified_model=True)
interface.eval_model('test', on_robustified_model=True)

In [ ]:
interface.generate_LPs()
interface.print_LPs()

In [ ]:
# alphas = [.5, .4, .3, .2, 1e-1, 1e-2, 1e-3, 1e-4, 8e-5, 5e-5, 2e-5, 1e-5, 5e-6, 1e-6, 5e-7, 1e-7, 7e-8, 5e-8, 2e-8, 1e-8, 5e-9, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15, 1e-16]
alphas = [1e-4]
on_robustified_model = True

for alpha in alphas:
    print('First evaluation...')
    (B_detect_ratio, A_detect_ratio), (B_LPs, A_LPs), (B_LPs_score, A_LPs_score) = interface.evaluate_algorithm_on_test_set(alpha, verbose=False, on_robustified_model=on_robustified_model)
    print('Second evaluation...')
    (B2_detect_ratio, _), (B2_LPs, _), (B2_LPs_score, _) = interface.evaluate_algorithm_on_test_set(alpha, verbose=False, dataset='train', on_robustified_model=on_robustified_model)
    # exp_info = [B2_detect_ratio, B_detect_ratio, A_detect_ratio]
    # print(alpha, exp_info)

In [ ]:
%matplotlib inline        
import matplotlib.pyplot as plt
import numpy as np

qr = '95'

B2LPs, BLPs, ALPs = np.array(B2_LPs), np.array(B_LPs), np.array(A_LPs) 
print(B2LPs.shape, BLPs.shape, ALPs.shape)

BLPs[BLPs=='benign'] = 1
BLPs[BLPs=='adversarial'] = 0
BLPs = BLPs.astype(np.int)
prob_BLPs = np.sum(BLPs, axis=0) / BLPs.shape[0]

B2LPs[B2LPs=='benign'] = 1
B2LPs[B2LPs=='adversarial'] = 0
B2LPs = B2LPs.astype(np.int)
prob_B2LPs = np.sum(B2LPs, axis=0) / B2LPs.shape[0]

ALPs[ALPs=='benign'] = 1
ALPs[ALPs=='adversarial'] = 0
ALPs = ALPs.astype(np.int)
prob_ALPs = np.sum(ALPs, axis=0) / ALPs.shape[0]

print('This indicates the portion of inputs to be judged as "benign"')
print(prob_B2LPs, 'train dataset (benign)')
print(prob_BLPs, 'test dataset (benign)')
print(prob_ALPs, 'test dataset (adversarial)')

# create plot
fig, ax = plt.subplots()
num_of_layers = 4
index = np.arange(num_of_layers)
bar_width = 0.2
opacity = 0.7

rects1 = plt.bar(index, prob_B2LPs, bar_width, alpha=opacity, color='b', label='Train Ben')
rects2 = plt.bar(index + bar_width, prob_BLPs, bar_width, alpha=opacity, color='g', label='Test Ben')
rects2 = plt.bar(index + 2 * bar_width, prob_ALPs, bar_width, alpha=opacity, color='r', label='Test Adv')

plt.xlabel('I-th layer')
plt.ylabel('Benign ratio')
plt.title('Benign ratio in different layers ('+qr+'qr)')
plt.xticks(index + bar_width, ('1', '2', '3', '4'))
plt.legend()

plt.tight_layout()
# plt.savefig('exp9_'+qr+'.png')
plt.show()

#-----------------------------------------------------------------------

def print_qrs(A, B, C, qrs):
    import numpy as np
    for qr in qrs:
        print(qr, 'qr:', np.percentile(A, qr), np.percentile(B, qr), np.percentile(C, qr))
    
B2LPs_score, BLPs_score, ALPs_score = np.array(B2_LPs_score), np.array(B_LPs_score), np.array(A_LPs_score) 
fig, _ = plt.subplots(nrows=1, ncols=4, figsize=(18, 3))
for i in range(BLPs_score.shape[1]):
    plt.subplot(1, 4, (i+1))
    B2LP_score, BLP_score, ALP_score = B2LPs_score[:,i], BLPs_score[:,i], ALPs_score[:,i]
    plt.plot(B2LP_score, np.arange(B2LP_score.shape[0]), 'o')
    plt.plot(BLP_score, np.arange(BLP_score.shape[0]), 'go')
    plt.plot(ALP_score, np.arange(ALP_score.shape[0]), 'ro')
    plt.legend(['Benign_train', 'Benign_test', 'Adv_test'], loc="lower right")
    plt.title('LP'+str(i+1))
    
    ##############################################
    # Compute 'mean' and '95q' in this block
    ##############################################
    print('LP'+str(i+1))
    print('Mean:', np.mean(B2LP_score), np.mean(BLP_score), np.mean(ALP_score))
    print_qrs(B2LP_score, BLP_score, ALP_score, [5, 50, 90, 95, 96, 97, 98, 99])
    ##############################################
    
# plt.savefig('exp9_01.png')
plt.show()

# 0. Correctly visualize B_train/B_test/A_test (DONE)
# 1. Write a snippet to generate '95q' scores (DONE)
# 2. Visualize the benign (or adversarial) ratio of samples by bar chart, comparing B train / B test / A test ()  
# 3. Based on two, also visualize using y ()
# 4. Record results of 1-3 ()

In [ ]:
def single_exp():
    interface.generate_model(num_of_epochs=5)
    interface.generate_LPs()
    (benign_detect_ratio, adversarial_detect_ratio), (benign_LPs, adversarial_LPs) = interface.evaluate_algorithm_on_test_set(alpha, verbose=False)
    exp_info = [benign_detect_ratio, adversarial_detect_ratio]
    return exp_info    

def exp(num_of_trails, file_name):
    cumulative_exp_info = []
    
    import csv 
    for i in range(num_of_trails):
        print('trail', (i+1))
        single_exp_info = single_exp()
        cumulative_exp_info.append(single_exp_info)

        with open(file_name, mode='a', newline='') as file:
            writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(single_exp_info)
        file.close()

        print(single_exp_info)
        
    return cumulative_exp_info

file_name = 'test.csv'
# cumulative_exp_info = exp(40, file_name)

In [ ]:
print('all pass')